In [34]:
##################### CODE DIFFERENCE UTILITY #####################
from turtle import up
import code_diff as cd

from code_diff.gumtree import ops

def _subtrees(script):
    subtrees = {}
    for action in script:
        if not isinstance(action, (ops.Insert, ops.Move)): continue
        target, node, position = action.target_node, action.node, action.position

        if isinstance(action, ops.Insert):
            _, text = node
            insert_content = text if text is not None else action.insert_id
            if isinstance(insert_content, int):
                continue
            subtrees[action.insert_id] = (insert_content)
            # target_id = target.node_id
            # if target_id not in subtrees: subtrees[target_id] = []
            # else: subtrees[target_id] = []
            # subtrees[target_id].insert(position, insert_content)
        elif isinstance(action, ops.Move):
            insert_content = node
            # subtrees[str(action.move_id)] = (insert_content)

        # if hasattr(target, "node_id"):
        #     target_id = target.node_id
        #     if target_id not in subtrees: subtrees[target_id] = []
        #     else: subtrees[target_id] = []
        #     subtrees[target_id].insert(position, insert_content)
    
    return subtrees

def _serialize_tree(subtrees, node_id):
    result = []
    stack  = [node_id]

    while len(stack) > 0:
        element = stack.pop(0)
        if isinstance(element, int):
            # element = str(element)
            elem = subtrees.get(element, [])
            if (elem != []):
                result.append(elem)
            else:
                stack = subtrees.get(element, []) + stack
        else:
            result.append(element)

    return result

def flatten_script(script):
    result_script = []
    subtrees = _subtrees(script)

    for action in script:
        if isinstance(action, ops.Insert):
            if not hasattr(action.target_node, "node_id"): continue # Ignore because we flatten
            new_node = _serialize_tree(subtrees, action.insert_id)
            result_script.append(ops.Insert(action.target_node, new_node, position = action.position, insert_id=action.insert_id))
        elif isinstance(action, ops.Move) and hasattr(action.target_node, "node_id"):
            result_script.append(ops.Delete(action.node))
        else:
            result_script.append(action)

    return result_script

def synthesize_rewrite_script(script):
    # Flatten the script: Build and parse the subtrees that are inserted or moved
    flat_script = flatten_script(script)

    # Generate new actions of the form (replace_span, token_seq)
    # You can transform the source by replacing each span with the token sequence 
    result = []
    for action in flat_script:
        target_node = action.target_node
        if isinstance(action, ops.Insert):
            # if action.position == len(target_node.children):
            target = target_node
            while (target.children == []):
                target = target.parent
            # if (target_node.children == []): 
            #     predecessor = target_node.parent
            #     (start_line, start_pos), (end_line, end_pos) = predecessor.position[1], predecessor.position[1]
            # else:
            predecessor = target
            (start_line, start_pos), (end_line, end_pos) = predecessor.position[1], predecessor.position[1]
            
            # else:
            #     if (target_node.children == []): # what I want is the line number only not column offset
            #         predecessor = target_node.parent
            #         (start_line, start_pos), (end_line, end_pos) = predecessor.position[1], predecessor.position[1]
            #     else:
            #         predecessor = target_node.children[action.position]
            #         (start_line, start_pos), (end_line, end_pos) = predecessor.position[1], predecessor.position[1]
            if (action.node == []): continue
            parent = target_node.parent
            while parent and parent.__class__.__name__ != 'ASTNode':
                # print(parent.__class__.__name__)
                parent = parent.parent

            print(parent.type)
            insertOperation = f'Insert: {(target_node).type} {action.node}: parent [{parent.type}]'
            # print(target_node.__class__.__name__)
            result.append(((start_line, start_pos, end_line, end_pos), [insertOperation]))
        elif isinstance(action, ops.Update):
            (start_line, start_pos), (end_line, end_pos) = target_node.position
            updateOperation = f'Update: {(target_node).type} from [{target_node.text}] to [{action.value}]'

            result.append(((start_line, start_pos, end_line, end_pos), [updateOperation]))
        elif isinstance(action, ops.Delete):
            (start_line, start_pos), (end_line, end_pos) = target_node.position
            deleteOperation = f'Delete: {(target_node).type}'
            result.append(((start_line, start_pos, end_line, end_pos), [deleteOperation]))
    return result

###################################################################################


In [33]:
output = cd.difference(
    '''
        expected = datetime . datetime ( 1970 , 1 , 1 , 1 , 0 , 23 , 0 , datefmt . localtz )
    ''',
    '''
        expected = datetime . datetime . fromtimestamp ( 23 , datefmt . localtz )
    ''',
lang = "python")



s = output.edit_script()
# s = synthesize_rewrite_script(s)
# print((s))

# Open the file in write mode
# with open('O:\DriveFiles\GP_Projects\Bug-Repair\Q-A\MyMutpy\output.txt', 'w') as file:
#     for action in s:
#         (start_line, start_pos, end_line, end_pos), new_content = action
#         file.write(f"{start_line} {start_pos} {end_line} {end_pos} {new_content}\n")

import json

def read_json_file(file_path):
    with open(file_path, 'r') as f:
        line_content = [json.loads(line) for line in f.readlines()]
    return line_content
data = read_json_file('O:\DriveFiles\GP_Projects\ctssb_data_1M\ctssb_data_1M/file-0.jsonl/file-0.jsonl')


import ast

print(data[111]["before"])
print(data[111]["after"])
# print(data[1]["comodified"])
# print(data[41]["edit_script"])
# print(data[631]["likely_bug"])

# s = ast.dump(ast.parse(data[31]["after"]), indent=4)
# print(s)
print("-------------------------------------")
b = "if (x == 1 and y == 2): pass"
a = """t = 2
if (x == 1 and z == 3):
 pass"""

diff = cd.difference(b, a, lang="python")
# print(diff.edit_script())
actions = synthesize_rewrite_script(diff.edit_script())
for action in actions:
    print(action)

print(ast.dump(ast.parse(b), indent=4))

except XMPPError : log . debug ( 'Could not retrieve vCard for %s' % jid )
except XMPPError : log . debug ( 'Could not retrieve vCard for %s' % self . xmpp . boundjid . bare )
-------------------------------------
module
module
module
((0, 28, 0, 28), ["Insert: assignment ['t']: parent [module]"])
((0, 28, 0, 28), ["Insert: assignment ['=']: parent [module]"])
((0, 28, 0, 28), ["Insert: assignment ['2']: parent [module]"])
((0, 15, 0, 16), ['Update: identifier from y to z'])
((0, 20, 0, 21), ['Update: integer from 2 to 3'])
Module(
    body=[
        If(
            test=BoolOp(
                op=And(),
                values=[
                    Compare(
                        left=Name(id='x', ctx=Load()),
                        ops=[
                            Eq()],
                        comparators=[
                            Constant(value=1)]),
                    Compare(
                        left=Name(id='y', ctx=Load()),
                        ops=[
             

In [13]:
from nltk.translate.bleu_score import sentence_bleu
reference = data[111]["before"].split()
# reference = [['this', 'is', 'a', 'test'], ['this', 'is' 'test']]
# candidate = ['this', 'is', 'a', 'gteet']
candidate = data[111]["after"].split()
score = sentence_bleu(reference, candidate, weights= [1])
print(score)

0.22727272727272727


In [17]:
from keyword import iskeyword


string_test = "except XMPPError : log . debug ( 'Could not retrieve vCard for %s' % jid )"

keyword_index = 0
attribute_index = 0

splitted = string_test.split()
for i, s in enumerate(splitted):
    if iskeyword(s):
        splitted[i]
        print(s)

except
not
for


In [31]:
import code_tokenize as ctok
# from tokenizer import tokenize
from nltk.tokenize import word_tokenize  
tok = word_tokenize(string_test)
# res = tokenize(string_test)
# res2 = []
# for r in res:
#     print(r.txt)
#     res2.append(r)
# res3 = " ".join(res2)
res = " ".join(tok)
print(tok)

['except', 'XMPPError', ':', 'log', '.', 'debug', '(', "'Could", 'not', 'retrieve', 'vCard', 'for', '%', 's', "'", '%', 'jid', ')']
